## Challenge 2: Aurora Bay RAG System
**Objective:** Program a Retrieval Augmented Generation (RAG) system using BigQuery Vector Search and Vertex AI.

**The Problem:** Large Language Models (LLMs) like Gemini are powerful, but they do not know about private or fictitious data. The Solution: We will inject "Aurora Bay" proprietary data into the model's context window to ground its answers in fact.

#### Environment Setup & Permissions
(Run this first to ensure the Service Account can actually talk to Vertex AI)

In [3]:
import subprocess
import time
import os

# --- CONFIGURATION ---
PROJECT_ID = "qwiklabs-gcp-03-ba43f2730b93" # <--- Check this matches!
REGION = "us-central1"

print("--- 0. Checking & Granting Permissions ---")
# 1. Get the BigQuery Service Account Email
# We derived this from your error logs earlier
SERVICE_ACCOUNT = "bqcx-281600971548-ntww@gcp-sa-bigquery-condel.iam.gserviceaccount.com"

# 2. Grant the Role
# We use gcloud to force-add the 'Vertex AI User' role to the Service Account
print(f"Granting 'roles/aiplatform.user' to {SERVICE_ACCOUNT}...")
command = f"gcloud projects add-iam-policy-binding {PROJECT_ID} --member='serviceAccount:{SERVICE_ACCOUNT}' --role='roles/aiplatform.user'"

# Execute the shell command
try:
    result = subprocess.run(command, shell=True, capture_output=True, text=True)
    if result.returncode == 0:
        print("✅ Permissions updated successfully.")
    else:
        print(f"⚠️ Notice: {result.stderr}")
except Exception as e:
    print(f"❌ Setup Error: {e}")

print("⏳ Waiting 10 seconds for IAM propagation...")
time.sleep(10)
print("✅ Environment Ready.")

--- 0. Checking & Granting Permissions ---
Granting 'roles/aiplatform.user' to bqcx-281600971548-ntww@gcp-sa-bigquery-condel.iam.gserviceaccount.com...
✅ Permissions updated successfully.
⏳ Waiting 10 seconds for IAM propagation...
✅ Environment Ready.


#### Setup & Baseline Test
(This demonstrates why we need RAG. The model should fail here.)

In [2]:
import vertexai
from google.cloud import bigquery
from vertexai.generative_models import GenerativeModel
import time

# --- CONFIGURATION ---
# We use 'us-central1' for all resources to ensure connectivity
PROJECT_ID = "qwiklabs-gcp-03-ba43f2730b93"
REGION = "us-central1"
DATASET_ID = "aurora_bay_rag"
CONNECTION_ID = "us-central1.vertex-ai-conn"
CSV_URI = "gs://labs.roitraining.com/aurora-bay-faqs/aurora-bay-faqs.csv"

# Initialize Clients
client = bigquery.Client(project=PROJECT_ID, location=REGION)
vertexai.init(project=PROJECT_ID, location=REGION)

print(f"--- 📉 BASELINE TEST: GENERIC MODEL (NO RAG) ---")
print("Asking Gemini about 'Aurora Bay' without providing any data...")

model = GenerativeModel("gemini-2.5-flash")
# We ask a specific question about the fictitious town
baseline_response = model.generate_content("What are the opening hours for the recycling center in Aurora Bay, Alaska?")

print(f"Generic Model Answer:\n'{baseline_response.text}'")
print("-" * 50)
print("OBSERVATION: The model likely hallucinated or said it didn't know. Now let's fix this with RAG.")

--- 📉 BASELINE TEST: GENERIC MODEL (NO RAG) ---
Asking Gemini about 'Aurora Bay' without providing any data...
Generic Model Answer:
'I'm having trouble finding a specific recycling center for a location named "Aurora Bay, Alaska." Aurora Bay doesn't appear to be a recognized populated place or municipality with its own dedicated recycling facility.

It's possible that:

1.  **Aurora Bay is a geographical feature** and residents would use services in a nearby town.
2.  **You might be referring to a different town** or a specific, perhaps private, facility.

**Could you clarify which town or larger area you're referring to?**

If you can provide the name of a nearby city or borough (e.g., Homer, Kenai, Seward, Anchorage), I can try to find information for their local waste and recycling facilities.'
--------------------------------------------------
OBSERVATION: The model likely hallucinated or said it didn't know. Now let's fix this with RAG.


### Step 1: Infrastructure & Data Ingestion
We will create a BigQuery dataset and load the proprietary FAQ data from Cloud Storage.

**Source:** `gs://labs.roitraining.com/.../aurora-bay-faqs.csv`

**Destination:** `aurora_bay_rag.faqs_raw`

In [4]:
print(f"--- 1. Setting up Dataset & Loading Data ---")

# 1. Create Dataset (in us-central1)
dataset = bigquery.Dataset(f"{PROJECT_ID}.{DATASET_ID}")
dataset.location = REGION
client.create_dataset(dataset, exists_ok=True)

# 2. Define Table & Schema
# We explicitly define the schema to ensure columns are named 'question' and 'answer'
table_ref = client.dataset(DATASET_ID).table("faqs_raw")
job_config = bigquery.LoadJobConfig(
    schema=[
        bigquery.SchemaField("question", "STRING"),
        bigquery.SchemaField("answer", "STRING"),
    ],
    source_format=bigquery.SourceFormat.CSV,
    skip_leading_rows=1, # Skip the CSV header
    write_disposition=bigquery.WriteDisposition.WRITE_TRUNCATE
)

# 3. Load from GCS
load_job = client.load_table_from_uri(CSV_URI, table_ref, job_config=job_config)
load_job.result()
print(f"✅ Data Loaded. Row count: {load_job.output_rows}")

--- 1. Setting up Dataset & Loading Data ---
✅ Data Loaded. Row count: 50


### Step 2: Vectorization
To search the data, we must convert the text into Embeddings (arrays of numbers representing meaning).

**Remote Model:** Connects BigQuery to Vertex AI's text-embedding-004.

**Vector Table:** We generate embeddings for every FAQ row and store them in faq_vectors.

In [5]:
def run_query(sql):
    """Helper to run SQL in the correct region"""
    try:
        job = client.query(sql, location=REGION)
        return job.result()
    except Exception as e:
        print(f"❌ Error: {e}")
        return None

print(f"--- 2. Creating Remote Model ---")
# Creates the bridge to Vertex AI
create_model_sql = f"""
CREATE OR REPLACE MODEL `{PROJECT_ID}.{DATASET_ID}.embedding_model`
REMOTE WITH CONNECTION `{PROJECT_ID}.{CONNECTION_ID}`
OPTIONS (ENDPOINT = 'text-embedding-004');
"""
run_query(create_model_sql)
time.sleep(5) # Allow model to propagate

print("--- 3. Generating Vector Index ---")
# Converts Text -> Vectors
# We concatenate Question + Answer to give the search engine full context
index_sql = f"""
CREATE OR REPLACE TABLE `{PROJECT_ID}.{DATASET_ID}.faq_vectors` AS
SELECT
  base.question,
  base.answer,
  emb.ml_generate_embedding_result as embedding
FROM ML.GENERATE_EMBEDDING(
  MODEL `{PROJECT_ID}.{DATASET_ID}.embedding_model`,
  (
    SELECT question, answer, CONCAT('Question: ', question, ' Answer: ', answer) as content
    FROM `{PROJECT_ID}.{DATASET_ID}.faqs_raw`
  )
) as emb
JOIN `{PROJECT_ID}.{DATASET_ID}.faqs_raw` as base
ON emb.question = base.question;
"""
run_query(index_sql)
print("✅ Knowledge Base Vectorized.")

--- 2. Creating Remote Model ---
--- 3. Generating Vector Index ---
✅ Knowledge Base Vectorized.


### Step 3: The RAG Application
This function implements the "Retrieval Augmented Generation" flow:

**Retrieve:** Convert the user's query to a vector and find the top 3 nearest neighbors in BigQuery (VECTOR_SEARCH).

**Augment:** Combine the retrieved FAQ answers into a "Context Block".

**Generate:** Send the User Question + Context Block to Gemini to write the final answer.

In [8]:
def ask_aurora_bay(user_query):
    print(f"\nUser Query: '{user_query}'")

    # --- A. RETRIEVAL (BigQuery Vector Search) ---
    search_sql = f"""
    SELECT base.answer, (1 - distance) AS similarity_score
    FROM VECTOR_SEARCH(
        TABLE `{PROJECT_ID}.{DATASET_ID}.faq_vectors`, 'embedding',
        (
            SELECT ml_generate_embedding_result, '{user_query}' AS query
            FROM ML.GENERATE_EMBEDDING(
                MODEL `{PROJECT_ID}.{DATASET_ID}.embedding_model`,
                (SELECT '{user_query}' AS content)
            )
        ),
        top_k => 3 -- Fetch top 3 most relevant facts
    )
    ORDER BY similarity_score DESC;
    """
    results = run_query(search_sql)

    # --- B. AUGMENTATION (Context Construction) ---
    context_text = ""
    if results:
        for row in results:
            context_text += f"- {row.answer}\n"

    if not context_text:
        return "Sorry, I couldn't find that info."

    # --- C. GENERATION (Gemini) ---
    prompt = f"""
    You are the official Town Clerk of Aurora Bay.
    Answer the user's question using ONLY the provided Knowledge Base below.

    KNOWLEDGE BASE:
    {context_text}

    USER QUESTION: {user_query}
    """

    model = GenerativeModel("gemini-2.5-flash")
    response = model.generate_content(prompt)
    return f"Aurora Bot: {response.text}"

print("✅ RAG Agent Initialized.")

✅ RAG Agent Initialized.


### Step 4: Final Evaluation
We now run the same question that failed in the beginning.

*   **Before:** The model hallucinated or failed.
*   **After:** The model should retrieve the exact hours from our CSV.





In [9]:
print("--- 📈 RAG PERFORMANCE TEST ---")

# 1. The Question that failed earlier
q1 = "What are the opening hours for the recycling center?"
print(ask_aurora_bay(q1))

# 2. Another specific local question
q2 = "Who is the mayor?"
print(ask_aurora_bay(q2))

# 3. A location question
q3 = "Where is the town hall?"
print(ask_aurora_bay(q3))

--- 📈 RAG PERFORMANCE TEST ---

User Query: 'What are the opening hours for the recycling center?'
Aurora Bot: I apologize, but the provided Knowledge Base does not contain information about the opening hours for the recycling center. It only states that a recycling drop-off center is located behind the Public Works building.

User Query: 'Who is the mayor?'
Aurora Bot: The current mayor is Linda Greenwood.

User Query: 'Where is the town hall?'
Aurora Bot: The Town Hall is located at 100 Harbor View Road, in the center of Aurora Bay, close to the main harbor.
